## Trabajador experto en conocimiento

Un agente que responde preguntas y es un trabajador experto en conocimiento
En este caso lo voy a utilizar para el emprendimiento que estamos trabajando con mis dos socios, Estudio3. Estudio3.ar
El agente va a responder información especifica de Estudio3, que obtendra da la pagina web, y la solución debe ser de bajo costo, intentare hacerlo con Open IA y luego con algun modelo de codigo abierto.

Utilizará RAG (Retrieval Augmented Generation, generación aumentada de recuperación)

In [1]:
!pip install langchain pymupdf

In [2]:
!pip install google-auth-oauthlib


In [22]:
#replico la importacion usual

import os
import glob
from dotenv import load_dotenv
import gradio as gr

In [23]:
# imports de langchain, plotly y Chroma

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

from langchain.document_loaders import DirectoryLoader, TextLoader, PyMuPDFLoader
from langchain.text_splitter import CharacterTextSplitter

In [24]:
# Comienzo con OPENIA

MODEL = "gpt-4o-mini"
db_name = "vector_db"

In [25]:
# Levanto las keys desde .env

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

In [ ]:
## le agrego llamar al google drive de mi emprendimiento
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import pickle

# Archivo JSON de credenciales (asegúrate de que esté en el mismo directorio que tu código)
CLIENT_SECRET_FILE = "credentials.json"
SCOPES = ["https://www.googleapis.com/auth/drive"]

# Autenticación con OAuth 2.0
flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
creds = flow.run_local_server(port=0)

# Guardar credenciales para futuras sesiones
with open("token.pickle", "wb") as token:
    pickle.dump(creds, token)

# Construcción del servicio de Google Drive
drive_service = build("drive", "v3", credentials=creds)

# Listar archivos en Drive para verificar acceso
results = drive_service.files().list(pageSize=10, fields="files(id, name)").execute()
files = results.get("files", [])

if not files:
    print("No se encontraron archivos en Google Drive.")
else:
    print("Archivos en Drive:")
    for file in files:
        print(f"{file['name']} ({file['id']})")

In [29]:
#aqui le indico leer solo un archivo, es para probar y luego extender es un gogole slide. 
#habilite la API de google slide
import os
from googleapiclient.discovery import build
import pickle


with open("token.pickle", "rb") as token:
    creds = pickle.load(token)


slides_service = build("slides", "v1", credentials=creds)

# ID del archivo Google Slides
PRESENTATION_ID = "1FyLW3R2wcqPL3nnibzbtLp-h6Y6AJFefepBxaqPIE2s"


presentation = slides_service.presentations().get(presentationId=PRESENTATION_ID).execute()

# Establecer el folder path, se suma a la carpeta servicios
folder_path = "Estudio3-base/Servicios"
os.makedirs(folder_path, exist_ok=True)

# Ruta del archivo donde se guardará el texto
file_path = os.path.join(folder_path, "OnePager_Estudio3_Servicios.md")

# Extraer texto y guardarlo en el archivo
with open(file_path, "w", encoding="utf-8") as file:
    for slide in presentation.get("slides", []):
        file.write(f"Diapositiva {slide['objectId']}:\n")
        slide_text = []  # Lista para almacenar el texto de la diapositiva
        
        for element in slide.get("pageElements", []):
            if "shape" in element and "text" in element["shape"]:
                text_elements = element["shape"]["text"].get("textElements", [])
                for text_element in text_elements:
                    if "textRun" in text_element:
                        slide_text.append(text_element["textRun"]["content"].strip())

        # Escribir solo si hay texto en la diapositiva
        if slide_text:
            file.write("\n".join(slide_text) + "\n")

        file.write("-" * 50 + "\n")

print(f"Texto guardado en: {file_path}")

Texto guardado en: Estudio3-base/Servicios/OnePager_Estudio3_Servicios.md


In [31]:
## Le agrego que lea tambien PDF 

folders = glob.glob("Estudio3-base/*")

def add_metadata(doc, doc_type):
    doc.metadata["doc_type"] = doc_type
    return doc

text_loader_kwargs = {'encoding': 'utf-8'}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)

    # Cargar archivos .md
    md_loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    md_docs = md_loader.load()
    
    # Cargar archivos .pdf
    pdf_loader = DirectoryLoader(folder, glob="**/*.pdf", loader_cls=PyMuPDFLoader)
    pdf_docs = pdf_loader.load()

    # Agregar metadata y combinar documentos
    documents.extend([add_metadata(doc, doc_type) for doc in md_docs + pdf_docs])

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

print(f"Total number of chunks: {len(chunks)}")
print(f"Document types found: {set(doc.metadata['doc_type'] for doc in documents)}")


Created a chunk of size 6735, which is longer than the specified 1000
Created a chunk of size 2966, which is longer than the specified 1000
Created a chunk of size 8026, which is longer than the specified 1000


Total number of chunks: 41
Document types found: {'Presupuestos', 'Servicios', 'Estudio3', 'Founders'}


In [32]:
embeddings = OpenAIEmbeddings()


if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

# Crear almacén de vectores

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 41 documents


In [33]:
# Investiguemos los vectores

collection = vectorstore._collection
count = collection.count()

sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"Hay {count:,} vectores con {dimensions:,} dimensiones en el almacén de vectores")

Hay 41 vectores con 1,536 dimensiones en el almacén de vectores


In [34]:
# Chat 
llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

# Setear la memoria
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# Agrego la cantidad a revisar
retriever = vectorstore.as_retriever(search_kwargs={"k": 60})

# juntando todo: configure la cadena de conversación con GPT 3.5 LLM, el almacén vectorial y la memoria
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

/var/folders/8w/cpvgvw0n1c19hc1_clypdl000000gn/T/ipykernel_33579/3693541071.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


In [35]:
# set up a new conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# putting it together: set up the conversation chain with the GPT 4o-mini LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [36]:
# Wrapping that in a function

def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

In [37]:
# Configuración de la interfaz mejorada
view = gr.ChatInterface(
    fn=chat,
    title="🤖 Asistente Virtual de Estudio 3",
    description="Hola! Soy el asistente virtual de E3, ¿cómo podemos ayudarte hoy?",
    theme="soft",  # Tema con bordes redondeados y colores suaves
    chatbot=gr.Chatbot(height=400)  # Ajuste de altura del chat
)

# Estilos adicionales
css = """
.chatbot-container {
    font-family: 'Arial', sans-serif;
    background-color: #f8f8f8;
    border-radius: 10px;
    padding: 10px;
}
.gradio-title {
    color: #2a5d67;
    font-weight: bold;
    font-size: 1.5em;
}
"""

view.launch(share=True)




/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/components/chatbot.py:282: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(
/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/chat_interface.py:317: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://47e3ee144b72293133.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
